<a href="https://colab.research.google.com/github/mahima-c/AutoEncoder/blob/main/encoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense
from tensorflow.keras import backend as K

In [29]:
class Autoencoder:
  """
  Autoender represent deep convolution autoencoder with mirror image of encoder and decoder unit 
  """
  def __init__(self,input_shape,conv_filter,conv_kernal,conv_stride,latend_space_dim):
          self.input_shape=input_shape#[28,28,1]
          self.conv_filter=conv_filter#[2,4,8]
          self.conv_kernal=conv_kernal#[3,5,3]
          self.conv_stride=conv_stride#[1 ,2 ,2]
          self.latend_space_dim=latend_space_dim
          self.encoder=None
          self.decoder=None
          self.model=None

          self._num_conv_layer=len(conv_filter)
          self._shape_before_bottleneck = None
          self._build()


  def summary(self):
        self.encoder.summary()  

  def _build(self):  
        self._build_encoder()   
        # self._build_decoder()   
        # self._build_autoencoder()   


  def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers = self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self.encoder = Model(encoder_input, bottleneck, name="encoder")

  def _add_encoder_input(self):
            return Input(shape=self.input_shape, name="encoder_input")

  def _add_conv_layers(self,encoder_input):
        "create all conv layer in encoder"
        x=encoder_input
        for layer_index in range(self._num_conv_layer):
            x = self._add_conv_layer(layer_index, x)
        return x  
      # x is graph of layer
  def _add_conv_layer(self, layer_index, x): 
        """Add a convolutional block to a graph of layers, consisting of
            conv 2d + ReLU + batch normalization.
            """
        layer_number=layer_index+1    
        conv_layer=Conv2D(filters=self.conv_filter[layer_index],
                          kernel_size=self.conv_kernal[layer_index],
                          strides=self.conv_stride[layer_index],
                          padding='same',
                          name=f"encoder_conv_layer_{layer_number}"
                                      )  
        x = conv_layer(x) 
        x= ReLU(name=f"encoder_relu_{layer_number}")(x) 
        x = BatchNormalization(name=f"encoder_bn_{layer_number}")(x)
        return x
        
  def _add_bottleneck(self, x):
      """Flatten data and add bottleneck (Dense layer)."""
      self._shape_before_bottleneck = K.int_shape(x)[1:] 
        #for store shape before because that need in decoder part for mirror image[batch size,shape=7,7,18]
      x = Flatten()(x)
      x = Dense(self.latend_space_dim, name="encoder_output")(x)
      return x
          



In [30]:
if __name__ == "__main__":
    autoencoder = Autoencoder(
        input_shape=(28, 28, 1),
        conv_filter=(32, 64, 64, 64),
        conv_kernal=(3, 3, 3, 3),
        conv_stride=(1, 2, 2, 1),
        latend_space_dim=2
        
    )
    autoencoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_layer_1 (Conv2D (None, 28, 28, 32)        320       
_________________________________________________________________
encoder_relu_1 (ReLU)        (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_bn_1 (BatchNormaliza (None, 28, 28, 32)        128       
_________________________________________________________________
encoder_conv_layer_2 (Conv2D (None, 14, 14, 64)        18496     
_________________________________________________________________
encoder_relu_2 (ReLU)        (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_bn_2 (BatchNormaliza (None, 14, 14, 64)        256 